## 03 Feature Engineering

#### Import relevant libraries

#### Read processed dataset

#### Create new features

#### Check for covariance

#### Filter for training dataset

#### Pickle model_data file